# 닥터풋헬스 스마트스토어 고객 리뷰 수집
- 링크: https://smartstore.naver.com/foothealth/products/4868868795
- 수집할 데이터 요소: 
    1. 평점 (Rating): `rating`
    2. 아이디 (User ID): `user_id`
    3. 작성날짜 (Review Date): `review_date`
    4. 제품명 (Product Name): `product_name`
    5. 간단리뷰 (Brief Review): `brief_review`
    6. 리뷰유형 (Review Type): `review_type`
        - 이 변수는 '한달사용기', '재구매', 'BEST' 등의 값을 포함할 수 있다.
    7. 리뷰내용 (Review Content): `review_content`
    8. 추천수 (Number of Likes/Recommendations): `recommendation_count`

In [1]:
# -*- coding: utf-8 -*-
from datetime import datetime
import pandas as pd
import numpy as np
import time, random
import re
from tqdm.notebook import trange, tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
# Chrome Webdriver 불러오기
def chromeWebdriver():
    # 크롬 드라이버 자동 설치
    chrome_service = ChromeService(executable_path=ChromeDriverManager().install())
    
    # 드라이버 옵션 설정
    options = Options()
    options.add_experimental_option('detach', True) # 브라우저 바로 닫힘 방지
    options.add_experimental_option('excludeSwitches', ['enable-logging']) # 불필요한 메시지 제거
    options.add_argument('--headless') # 백그라운드 작업
    options.add_argument('--no-sandbox') # Bypass OS security model

    driver = webdriver.Chrome(service=chrome_service, options=options)
    
    return driver

In [3]:
# 스크래핑할 스마트스토어 주소 불러오기
url = "https://smartstore.naver.com/docsole/products/7381702448"
driver = chromeWebdriver()
driver.get(url)

SCROLL_PAUSE_SEC = 1

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 높이 초기화
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height: # 새로 업로드되는 페이지가 없다면
        break
    last_height = new_height
    
# 리뷰 버튼 클릭
driver.find_element(By.XPATH, r'//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]').click()
time.sleep(1)
# 최신순 정렬 클릭
driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[2]').click()
time.sleep(1)

# 리뷰 개수로 페이지 수 계산
num_reviews = driver.find_element(By.XPATH, r'//*[@id="content"]/div/div[2]/div[1]/div[2]/div[1]/a/strong').text 
num_reviews = int(num_reviews.replace(",", ""))
max_page = int(num_reviews//20 + 1)
print('리뷰 수: ',num_reviews,', 페이지 수: ', max_page)

리뷰 수:  743 , 페이지 수:  38


In [4]:
# 각 크롤링 결과 저장하기 위한 리스트 선언 
rating = []
user_id = []
review_date = []
product_info = []
review_text = []
recommendation_count = []
image_yn = []

# 페이지를 처음부터 끝까지, (20년도까지) 이동하면서 데이터 수집
page = 1
date = '2024.01.01'

while (page <= max_page) & ('20.01.01' <= date) :
    lis = driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul').find_elements(By.TAG_NAME, 'li') 
    print('● 현재 페이지:', page, ', 해당 페이지 리뷰수:', len(lis))

    for i in trange(1, len(lis) + 1):
        try: 
            rating.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/em').text)
            driver.implicitly_wait(1)
        except:
            rating.append(np.nan)
        try:    
            user_id.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/strong').text)
            driver.implicitly_wait(1)
        except:
            user_id.append(np.nan)
        try: 
            date = driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/span').text
            review_date.append(date)
            driver.implicitly_wait(1)
        except:
            review_date.append(np.nan)
        try:
            product_info.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[3]').text)
            driver.implicitly_wait(1)
        except:
            product_info.append(np.nan)
        try: 
            review_text.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[2]/div').text)
            driver.implicitly_wait(1)
        except:
            review_text.append(np.nan)
        try: 
            recommendation_count.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[2]/div/div/div/button/span').text)
            driver.implicitly_wait(1)
        except:
            recommendation_count.append(np.nan)
        try:
            driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[2]')
            image_yn.append('Y')
        except:
            image_yn.append('N')
            
    # click next button
    driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
    time.sleep(1)
    page += 1

driver.quit()
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])
df.head()

● 현재 페이지: 1 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

● 현재 페이지: 2 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

● 현재 페이지: 3 , 해당 페이지 리뷰수: 20


...

● 현재 페이지: 37 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

● 현재 페이지: 38 , 해당 페이지 리뷰수: 3


  0%|          | 0/3 [00:00<?, ?it/s]

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome-headless-shell=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x00000001012e04dc chromedriver + 4162780
1   chromedriver                        0x00000001012d8664 chromedriver + 4130404
2   chromedriver                        0x0000000100f2f9f8 chromedriver + 293368
3   chromedriver                        0x0000000100f77870 chromedriver + 587888
4   chromedriver                        0x0000000100f6b994 chromedriver + 539028
5   chromedriver                        0x0000000100f6b250 chromedriver + 537168
6   chromedriver                        0x0000000100fb5ab0 chromedriver + 842416
7   chromedriver                        0x0000000100f696bc chromedriver + 530108
8   chromedriver                        0x0000000100f6a930 chromedriver + 534832
9   chromedriver                        0x00000001012a5e08 chromedriver + 3923464
10  chromedriver                        0x00000001012

In [8]:
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])
df.head()

,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
0,24.01.03.,ryan****,5,타입: D타입(정렬은 바르나 아치가 낮은 평발) / 사이즈(좌/우): 290mm\n...,한달사용기하는 일이 하루에 5시간 이상 서서 일하는 업무라서 이런 저런 신발을 다 ...,Y,0
1,23.12.28.,hwle*****,5,타입: N타입(약한 평발) / 사이즈(좌/우): 265mm\n평소사이즈265mm평발...,한달사용기아이 발이 약간 평발인데 운동하고 걸을 때 편하다하네요,Y,0
2,23.12.28.,ryan****,5,타입: D타입(정렬은 바르나 아치가 낮은 평발) / 사이즈(좌/우): 290mm\n...,재구매발바닥이 3년정도 아팠습니다.\n정형외과를 다니면서 체외 충격파만 50회 이상...,Y,1
3,23.12.28.,ysk6***,5,타입: B타입(심한 평발) / 사이즈(좌/우): 245mm\n평소사이즈250mm평발...,한달사용기쓰기 전보다 확실히 발이 편해요,Y,1
4,23.12.26.,6317***,3,타입: D타입(정렬은 바르나 아치가 낮은 평발) / 사이즈(좌/우): 265mm\n...,만족하고 잘 사용하고있습니다~,Y,0


In [9]:
df.to_excel("data/Docsole.xlsx", index=False)

In [ ]:
page

In [ ]:
max_page

In [ ]:
len(review_date)

In [ ]:
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])

In [ ]:
df = pd.read_excel('data/Xsole.xlsx')
df.head()

In [ ]:
df.info()

In [ ]:
df.tail()